In [1]:
# 必要なライブラリをインポート
import torch
import torch.nn.functional as F
from transformers import GenerationConfig
from transformers import LlamaForCausalLM, LlamaTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "stored_output/checkpoint-2epoch"

/home/yosshstd/anaconda3/envs/unsloth_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# モデルとトークナイザーの読み込み
model = LlamaForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map='auto',
    quantization_config=None
)
tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH)

model.eval() # 推論モード設定

Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.34s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): ModulesToSaveWrapper(
      (original_module): Embedding(32000, 4096, padding_idx=0)
      (modules_to_save): ModuleDict(
        (default): Embedding(32000, 4096, padding_idx=0)
      )
    )
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Lin

In [3]:
generation_config = GenerationConfig(
    temperature=0.2,    # 出力の多様性を制御（低いほど決定論的）
    top_k=40,          # トップKのトークンのみを考慮
    top_p=0.9,         # 累積確率がこの値を超えるまでのトークンを考慮
    do_sample=True,    # ランダムサンプリングを有効化
    num_beams=1,       # ビーム探索の数
    repetition_penalty=1.2,  # 繰り返しを抑制
    max_new_tokens=900      # 生成する最大トークン数
)

In [4]:
# 生成したいアミノ酸配列の条件付け
input_text = "[Generate by Substrate] Substrate=<water>"
# トークン化
inputs = tokenizer(input_text, return_tensors="pt")

# テキスト生成
with torch.no_grad():
    outputs = model.generate(
        input_ids = inputs["input_ids"].to(device),
        attention_mask = inputs['attention_mask'].to(device),
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        generation_config = generation_config,
        return_dict_in_generate=True,
        output_attentions=False
    )

# 生成されたテキストをデコード
s = outputs["sequences"][0]
output = tokenizer.decode(s, skip_special_tokens=True)
output = output.replace("</s>", "")
print(output.replace(input_text, "").strip())

From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


Seq=<MKKEVCSVAFLKAVFAEFLATLIFVFFGLGSALKWPSALPTILQISLAFGLAIGTMAQALGPVSGGHINPAITLALLVGNQISLLRAVFYVVAQLVGAIAGAAILHEITPADSREGLAINKLHNETTTGQAVTVELFLTFQLVLCIFASDNRRNDNVGSPALSIGLSVTLGHLVGIYFTGCSMNPARSFAPAVIMTRFSHPEFNFDPDPLWR>


In [40]:
# 入力
input_text = "[Generate by Substrate] Substrate=<magnesium(2+)>"
input_text = "[Determine Substrate] Seq=<MTDMNIENRKLNRPASENDKQHKKVFPIEAEAFHSPEETLARLNSHRQGLTIEEASERLKVYGRNEVAHEQVPPALIQLLQAFNNPFIYVLIALAGVSFITDYWLPLRRGEETDLTGVLIILTMVSLSGLLRFWQEFRTNRAAQALKKMVRTTATVLRRGPGNIGAVQEEIPIEELVPGDVVFLAAGDLVPADVRLLASRDLFISQSILSGESLPVEKYDVMADVAGKDSEQLPDKDKSLLDLGNICLMGTNVTSGRAQAVVVATGSRTWFGSLAKSIVGTRTQTAFDRGVNSVSWLLIRFMLIMVPVVLLINGFSKGDWVEASLFALAVAVGLTPEMLPMIVSSNLAKGAIAMSRRKVIVKRLNAIQNFGAMDVLCTDKTGTLTQDNIFLEHHLDVSGVKSGRVLMLAWLNSSSQSGARNVMDRAILRFGEGRIAPSTKARFIKRDELPFDFVRRRVSVLVEDAQHGDRCLICKGAVEEMMMVATHLREGDRVVALTETRRELLLAKTEDYNAQGFRVLLIATRKLDGSGNNPTLSVEDETELTIEGMLTFLDPPKESAGKAIAALRDNGVAVKVLTGDNPVVTARICLEVGIDTHDILTGTQVEAMSDAELASEVEKRAVFARLTPLQKTRILQALQKNGHTVGFLGDGINDAPALRDADVGISVDSAADIAKESSDIILLEKDLMVLEEGVIKGRETFGNIIKYLNMTASSNFGNVFSVLVASAFIPFLPMLAIHLLIQNLMYDISQLSLPWDKMDKEFLRKPRKWDAKNIGRFMLWIGPTSSIFDITTFALMWYVFAANNVEAQALFQSGWFIEGLLSQTLVVHMLRTQKIPFIQSRATLPVLLTTGLIMAIGIYIPFSPLGAMVGLEPLPLSYFPWLVATLLSYCLVAQGMKRFYIKRFGQWF>"
# トークン化
inputs = tokenizer(input_text, return_tensors="pt")
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs['attention_mask'].to(device)
input_length = input_ids.shape[1] # 入力トークンの長さを取得

# --- generateメソッドの変更と追加 ---
# テキスト生成
with torch.no_grad():
    outputs = model.generate(
        input_ids = input_ids,
        attention_mask = attention_mask,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        generation_config = generation_config, # 既存の設定を使用
        return_dict_in_generate=True,
        output_scores=True, # <--- 各ステップのスコアを出力させる
        output_attentions=False # 既存の設定
    )

# 生成されたシーケンス全体 (プロンプト + 生成部分)
generated_ids_all = outputs.sequences[0]
# 生成された部分のみのIDを取得
generated_ids_only = generated_ids_all[input_length:]

# 各生成ステップでのロジット (スコア)
scores = outputs.scores # タプル形式: (ステップ1のロジット, ステップ2のロジット, ...)

# --- パープレキシティと Log(p) Per Token の計算 ---
total_log_prob = 0.0
num_generated_tokens = 0

if len(generated_ids_only) > 0 and scores is not None:
    # 各生成ステップについてループ
    for i in range(len(generated_ids_only)):
        # i番目のステップのロジットを取得 (batch_size=1を想定)
        step_logits = scores[i][0] # Shape: (vocab_size,)

        # ロジットを対数確率に変換 (log_softmax)
        step_log_probs = F.log_softmax(step_logits, dim=-1)

        # 実際に生成されたトークンのIDを取得
        generated_token_id = generated_ids_only[i]

        # そのトークンの対数確率を取得して加算
        token_log_prob = step_log_probs[generated_token_id].item()
        total_log_prob += token_log_prob
        num_generated_tokens += 1

        # EOSトークンで終了した場合の考慮 (EOS自体の確率は含める)
        if generated_token_id == tokenizer.eos_token_id:
            break # EOS以降のスコアは無関係な場合があるためループを抜ける

    if num_generated_tokens > 0:
        # トークンあたりの平均対数確率 (Log(p) Per Token)
        avg_log_prob = total_log_prob / num_generated_tokens
        # パープレキシティ (Perplexity)
        # avg_log_prob は負の値なので、-avg_log_prob は正の値 (平均負の対数尤度 NLL)
        perplexity = torch.exp(torch.tensor(-avg_log_prob)).item()
    else:
        avg_log_prob = float('-inf')
        perplexity = float('inf')
else:
    # 何も生成されなかった場合やスコアが得られなかった場合
    avg_log_prob = float('-inf') # または 0.0 や nan
    perplexity = float('inf')    # または 1.0 や nan
    num_generated_tokens = 0


# --- 既存のデコード処理と結果表示 ---
output_decoded = tokenizer.decode(generated_ids_all, skip_special_tokens=True)

# 正確に生成部分のみをデコードする
generated_part_decoded = tokenizer.decode(generated_ids_only, skip_special_tokens=True).replace("</s>", "").strip()

print(f"Input Text: {input_text}")
print("-" * 30)
print(f"Output Text: {generated_part_decoded}")
print("-" * 30)
print(f"Perplexity (PPL): {perplexity:.4f}")

Input Text: [Determine Substrate] Seq=<MTDMNIENRKLNRPASENDKQHKKVFPIEAEAFHSPEETLARLNSHRQGLTIEEASERLKVYGRNEVAHEQVPPALIQLLQAFNNPFIYVLIALAGVSFITDYWLPLRRGEETDLTGVLIILTMVSLSGLLRFWQEFRTNRAAQALKKMVRTTATVLRRGPGNIGAVQEEIPIEELVPGDVVFLAAGDLVPADVRLLASRDLFISQSILSGESLPVEKYDVMADVAGKDSEQLPDKDKSLLDLGNICLMGTNVTSGRAQAVVVATGSRTWFGSLAKSIVGTRTQTAFDRGVNSVSWLLIRFMLIMVPVVLLINGFSKGDWVEASLFALAVAVGLTPEMLPMIVSSNLAKGAIAMSRRKVIVKRLNAIQNFGAMDVLCTDKTGTLTQDNIFLEHHLDVSGVKSGRVLMLAWLNSSSQSGARNVMDRAILRFGEGRIAPSTKARFIKRDELPFDFVRRRVSVLVEDAQHGDRCLICKGAVEEMMMVATHLREGDRVVALTETRRELLLAKTEDYNAQGFRVLLIATRKLDGSGNNPTLSVEDETELTIEGMLTFLDPPKESAGKAIAALRDNGVAVKVLTGDNPVVTARICLEVGIDTHDILTGTQVEAMSDAELASEVEKRAVFARLTPLQKTRILQALQKNGHTVGFLGDGINDAPALRDADVGISVDSAADIAKESSDIILLEKDLMVLEEGVIKGRETFGNIIKYLNMTASSNFGNVFSVLVASAFIPFLPMLAIHLLIQNLMYDISQLSLPWDKMDKEFLRKPRKWDAKNIGRFMLWIGPTSSIFDITTFALMWYVFAANNVEAQALFQSGWFIEGLLSQTLVVHMLRTQKIPFIQSRATLPVLLTTGLIMAIGIYIPFSPLGAMVGLEPLPLSYFPWLVATLLSYCLVAQGMKRFYIKRFGQWF>
------------------------------
Output Text: Substra

In [ ]:
# 予測したいアミノ酸配列
input_text = "[Determine Substrate] Seq=<MTTLRKLPLALAVAAGVLTTQAMAVDFHGYARSGIGWTGSGGDQQCFKATGASSKYRLGNECETYAEIKLGQEVWKEADKSFYFDSNIAYSINQEDDWESTSPAFREANIQAKNLIDSLPGATMWAGKRYYQRHDVHMIDFYYWDISGPGAGLQDVDLAFGKLSFAATRSSEAGGSNSFASDNIRDYTSSTANDVFDIRLAELNTNPNGVLELGVDYGRANARDGYHLADDATKDGWMFTGEHTQSIWGGFNKFVVQYATDSMTSWNTHSCKDTNDAPNAFSISDDAGNLIHHVIDPDKTRVMSLMGMYQTYEKTDLYSHSRLLDLNNIMVTGAGEKIGIPDITSCGTNRNFENNTVSYDRSVLRRILDVE>"
# トークン化
inputs = tokenizer(input_text, return_tensors="pt")

# テキスト生成
with torch.no_grad():
    outputs = model.generate(
        input_ids = inputs["input_ids"].to(device),
        attention_mask = inputs['attention_mask'].to(device),
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        generation_config = generation_config,
        return_dict_in_generate=True,
        output_attentions=False
    )

# 生成されたテキストをデコード
s = outputs["sequences"][0]
output = tokenizer.decode(s, skip_special_tokens=True)
output = output.replace("</s>", "")
print(output.replace(input_text, "").strip())

Substrate=<fluoride>


In [1]:
from Bio.PDB import PDBParser

def get_average_plddt(pdb_file):
    parser = PDBParser()
    structure = parser.get_structure("protein", pdb_file)
    plddts = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if "CA" in residue:  # CA原子が存在する場合(アミノ酸残基)
                    plddts.append(residue["CA"].get_bfactor()) # B-factorがpLDDT
    return sum(plddts) / len(plddts)

predicted_pdb_file = "output_structure.pdb"  # AlphaFold2 の出力 PDB ファイル
avg_plddt = get_average_plddt(predicted_pdb_file)
print(f"Average pLDDT (generated): {avg_plddt:.3f}")

Average pLDDT (generated): 0.903


/home/yosshstd/anaconda3/envs/unsloth_env/lib/python3.12/site-packages/Bio/PDB/PDBParser.py:384: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 5367
  warnings.warn(
